In [1]:
%pip install nltk
%pip install tweepy
%pip install pandas
%pip install wordcloud
%pip install textblob
%pip install seaborn
%pip install poyraz
%pip install spacy
%pip install sklearn
%pip install zemberek-python
%pip install trnltk
%pip install git+https://github.com/emres/turkish-deasciifier.git
%pip install transformers datasets
%pip install torch
%pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated pa

ERROR: Could not find a version that satisfies the requirement poyraz (from versions: none)
ERROR: No matching distribution found for poyraz


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/emres/turkish-deasciifier.git to c:\users\cemke\appdata\local\temp\pip-req-build-ismsiig6
  Resolved https://github.com/emres/turkish-deasciifier.git to commit 665154c734b09485c3d11ce0038cd121dd109594
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may 

  Running command git clone --filter=blob:none --quiet https://github.com/emres/turkish-deasciifier.git 'C:\Users\cemke\AppData\Local\Temp\pip-req-build-ismsiig6'


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
#!/usr/bin/env python
#-*-coding:utf-8-*-
from zemberek import (
    TurkishSpellChecker,
    TurkishSentenceNormalizer,
    TurkishSentenceExtractor,
    TurkishMorphology,
    TurkishTokenizer
)
import logging
import tweepy
import json
import csv
import sys
import time
import re
import warnings
import string
import pandas as pd
import ast
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import Stream
from wordcloud import WordCloud
from textblob import TextBlob
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from spacy.tokenizer import Tokenizer
from spacy.lang.tr import Turkish
from trnlp import TrnlpWord
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer, TextClassificationPipeline
from transformers import pipeline
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings('ignore')

2023-01-11 22:55:17,290 - numexpr.utils - INFO
Msg: Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.

2023-01-11 22:55:17,291 - numexpr.utils - INFO
Msg: NumExpr defaulting to 8 threads.



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cemke\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
class getTweets:
    def __init__(self,ckey,csecret,atoken,asecret):
        self.api=None
        self.consumer_key = ckey
        self.consumer_secret = csecret
        self.access_token = atoken
        self.access_token_secret = asecret
        
    def setAuth(self):
        try:
            self.auth = tweepy.OAuthHandler(self.consumer_key, self.consumer_secret)
            self.auth.set_access_token(self.access_token, self.access_token_secret)
            self.api = tweepy.API(self.auth)            
            print ("Bağlantı Yapıldı!")                           
        except tweepy.TweepError as err:
            print('Error: {}'.format(err))   
    
    def toDataFrame(tweets):
        try:
            print ("Veriler Kayıt Ediliyor...")
            pd.set_option('display.max_rows', None)
            pd.set_option('display.max_columns', None)
            pd.set_option('display.width', None)
            pd.set_option('display.max_colwidth', None)
            DataSet = pd.DataFrame()

            DataSet['Kullanıcı_Adı'] = [tweet.user.name for tweet in tweets]
            DataSet['Tweet_Tarih'] = [tweet.created_at for tweet in tweets]   
            #DataSet['Tweet_Id'] = [tweet.id for tweet in tweets]
            DataSet['Tweet'] = [tweet.text for tweet in tweets]
            DataSet['Tweet_Degistirilmis'] = [tweet.text for tweet in tweets]
            DataSet['Tweet_Ceviri']=[tweet.text for tweet in tweets]            
            #DataSet['Kullanıcı_Id'] = [tweet.user.id for tweet in tweets]  
                
            #DataSet['Dil'] = [tweet.lang for tweet in tweets]
            print ("Veriler Kayıt Edildi!")
            return DataSet
        except Exception as err:
            print('Error: {}'.format(err))
    
    def searchTweet(self,aranan):
        self.setAuth()
        try:                      
            print ("Kelimeye Uygun Tweetler Sorgulanıyor...")
            cursor = tweepy.Cursor(self.api.search_tweets, q=aranan, result_type="new", lang='tr')
            results=[]
            for item in cursor.items(10):
                results.append(item)
            print ("Kelimeye Uygun Tweetler Sorgulandı!")
            DataSet = getTweets.toDataFrame(results)  
            getTweets.TweetClear(DataSet)
            getTweets.SentenceNormalization(DataSet)
            getTweets.TweetTranslate(DataSet)
            getTweets.KelimeAyristirma(DataSet) 
            DataSet.to_csv('tw.csv',index=False)
        except Exception as err:
            print('Error: {}'.format(err))
        finally:
            getTweets.TextblobSentiment(DataSet)

    def TextblobSentiment(DataSet):
        try:
            print ("Duygu Analizi Yapılıyor...")
            Veri=DataSet
            Veri["Tür"] = Veri["Tweet_Ayrı"].apply(getTweets.getPolarity)
            Veri["Yoğunluk"] = Veri["Tweet_Ayrı"].apply(getTweets.getSubjectivity)
            Veri.to_csv('textblob_eng_duygu.csv',index=False)
            print("Duygu Analizi Yapıldı!")                        
        except Exception as err:
            print('Error: {}'.format(err)) 
            
    """def TrSentiment(DataSet):
        try:
            print("Türkçe Duygu Analizi Yapılıyor...")
            for example,index in DataSet["Tweet_Degistirilmis"]:
                DataSet["TR_A"][index] = simple_token(example, sw=stopwords)
            print("Türkçe Duygu Analizi Yapıldı!")
        except Exception as err:
            print('Error: {}'.format(err)) """
                     
    def TweetTranslate(DataSet):
        try:
            print ("Tweetler Çeviriliyor...")
            for yazi in range(len(DataSet["Tweet_Degistirilmis"])):
                Ceviri_Blob=TextBlob(DataSet["Tweet_Degistirilmis"][yazi])
                Eng_Blob=Ceviri_Blob.translate(from_lang='tr', to='en')  
                DataSet["Tweet_Ceviri"][yazi]=Eng_Blob
                DataSet["Tweet_Ceviri"][yazi]=DataSet["Tweet_Ceviri"][yazi].lower()
            print ("Tweetler Çevirildi!")                      
            return DataSet        
        except Exception as err:
            print('Error: {}'.format(err))
            
    def TweetClear(DataSet):
        try:
            print ("Tweetler Temizleniyor...")
            pattern = re.compile('http[s]?://(?:[a-zA-Z0-9ğüşöçıİĞÜŞÖÇ]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F]))+|#[a-zA-Z0-9ğüşöçıİĞÜŞÖÇ]+|$[a-zA-Z0-9ğüşöçıİĞÜŞÖÇ]+|@[a-zA-Z0-9ğüşöçıİĞÜŞÖÇ_]+|[,.^_$*%-;鶯!?’"“”:=|…#]+|\n+|RT+|amp')
            for i in range(len(DataSet["Tweet_Degistirilmis"])):        
                DataSet["Tweet_Degistirilmis"][i]=pattern.sub(' ', DataSet["Tweet_Degistirilmis"][i])    
                DataSet["Tweet_Degistirilmis"][i]=DataSet["Tweet_Degistirilmis"][i].lower()              
            DataSet["Tweet_Degistirilmis"]=DataSet["Tweet_Degistirilmis"].str.replace("[^\w\s]","")
            print ("Tweetler Temizlendi!")
            return DataSet
        except Exception as err:
            print('Error: {}'.format(err))
            
    def SentenceNormalization(DataSet):
        try:
            morphology = TurkishMorphology.create_with_defaults()
            normalizer = TurkishSentenceNormalizer(morphology)

            start = time.time()
            for example,index in DataSet["Tweet_Degistirilmis"]:
                 DataSet["Tweet_Degistirilmis"][index]= normalizer.normalize(example)
            return DataSet
        except Exception as err:
            print('Error: {}'.format(err))
            
    def KelimeAyristirma(DataSet):
        try:
            stop = stopwords.words('english')
            DataSet["Tweet_Ayrı"] = DataSet["Tweet_Ceviri"].str.split()
            DataSet["Tweet_Ayrı"] = DataSet["Tweet_Ayrı"].apply(lambda x: [item for item in x if item not in stop])
            DataSet["Tweet_Ayrı"] = DataSet["Tweet_Ayrı"].str.join(" ")
            return DataSet
        except Exception as err:
            print('Error: {}'.format(err))
    
    def ToxicSentiment(DataSet):
        try:
            model_path ="martin-ha/toxic-comment-model"
            tokenizer=AutoTokenizer.from_pretrained(model_path)
            model=AutoModelForSequenceClassification.from_pretrained(model_path)
            pipeline=TextClassificationPipeline(model=model, tokenizer=tokenizer)
            text="bu cümle toksik bir cümle değil amınakoyayım"
            b_text=TextBlob(text)
            e_text=b_text.translate(from_lang='tr', to='en')  
            sonuc=pipeline(str(e_text))
            if sonuc[0]['label']=="toxic":
                print ("Toxic")
            else:
                print ("Toxic değil")        
        except Exception as err:
            print('Error: {}'.format(err))
        
    def getSubjectivity(DataSet):
        try: 
            sub=TextBlob(DataSet).sentiment.subjectivity               
            if sub >=0.75:
                return "Kesinlikle"
            elif sub >=0.5 and sub <0.75:
                return "Yüksek İhtimal"
            elif sub >=0.25 and sub <0.50:
                return "Düşük İhtimal"
            else:
                return "Kesin Değil"
        except Exception as err:
            print('Error: {}'.format(err))
            
    def getPolarity(DataSet):
        try:
            pol=TextBlob(DataSet).sentiment.polarity
            if pol>0.1:
                return "Pozitif"
            elif pol<-0.1:
                return "Negatif"
            else:
                return "Nötr"
        except Exception as err:
            print('Error: {}'.format(err))

In [4]:
tw = getTweets( 
ckey = 'o7qidRODRWm5u07PtAWrCpSqf',
csecret = '1Yn60OYMJl5kRCSM0XQ4Lskxh6hjM4VMz51OdsS6dM9DQuQ4MV',
atoken = '1315792402234441730-Dt7AydgVGYI5VN9RRQfnR8Endh0WAc',
asecret = 'jhfiDfB53Z27EqHfvfKbNdo0w2SWLqyyoAkILqgSoaGMK'
)
aranan=input("Aranacak Kelimeyi Girin: ")
tw.searchTweet(aranan)

Bağlantı Yapıldı!
Kelimeye Uygun Tweetler Sorgulanıyor...
Kelimeye Uygun Tweetler Sorgulandı!
Veriler Kayıt Ediliyor...
Veriler Kayıt Edildi!
Tweetler Temizleniyor...
Tweetler Temizlendi!
2023-01-11 22:56:20,790 - zemberek.morphology.turkish_morphology - INFO
Msg: TurkishMorphology instance initialized in 7.054992198944092

Error: too many values to unpack (expected 2)
Tweetler Çeviriliyor...
Tweetler Çevirildi!
Duygu Analizi Yapılıyor...
Duygu Analizi Yapıldı!


In [5]:
Veri=pd.read_csv('textblob_eng_duygu.csv')
Veri.head()

,Kullanıcı_Adı,Tweet_Tarih,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Ayrı,Tür,Yoğunluk
0,Zühtü,2023-01-11 19:56:13+00:00,"RT @HaberMujdeli: Meral Akşener yine bir gün hayallerini anlatırken.. \n\nDeliriyorsunuz, kuduruyorsunuz Erdoğan gibi olamadığınız için 😌\n\n h…",meral akşener yine bir gün hayallerini anlatırken deliriyorsunuz kuduruyorsunuz erdoğan gibi olamadığınız için h,"meral akşener again one day while telling your dreams, you get crazy, you can not be like erdogan h.","meral akşener one day telling dreams, get crazy, like erdogan h.",Negatif,Kesinlikle
1,Umut Can,2023-01-11 19:56:12+00:00,"Yine gel akşamüstü , gece sabaha varmadan.",yine gel akşamüstü gece sabaha varmadan,come again after the morning in the morning,come morning morning,Nötr,Kesin Değil
2,Şeyda,2023-01-11 19:56:12+00:00,RT @OgretmnEtkinlik: 📢 @FOXTurkiye\n Bu akşam yine 100 bin öğretmen dedi. Gündemde olmaya devam ediyoruz. Cumhuriyetin 100. Yılına özel 100…,bu akşam yine bin öğretmen dedi gündemde olmaya devam ediyoruz cumhuriyetin yılına özel,"we continue to be on the agenda of a thousand teachers again this evening, we continue to be on the agenda.","continue agenda thousand teachers evening, continue agenda.",Nötr,Kesin Değil
3,ℳᎯᏉℐ 💙,2023-01-11 19:56:12+00:00,RT @arto_sagatelyan: Yine gözüme çarptı \nNe zaman rastlarsam sinirlerim zıplıyor 😠😡🤬 https://t.co/c5XbM0Ubnx,yine gözüme çarptı ne zaman rastlarsam sinirlerim zıplıyor,"it hit me again, whenever i come across my nerves jump","hit again, whenever come across nerves jump",Nötr,Kesin Değil
4,Şükru Bircan🇹🇷🇵🇸,2023-01-11 19:56:12+00:00,"RT @demir_erdem: Sevdamız var diyerek yine yollara düştük…\n\nBugün,\n\n📍Kısıklı \n📍Salacak\n📍Küplüce Mahallelerimizde ziyaretlerimizi gerçekleşt…",sevdamız var diyerek yine yollara düştük bugün kısıklı salacak küplüce mahallelerimizde ziyaretlerimizi gerçekleşt,we have fell on the roads again by saying that we have love today.,fell roads saying love today.,Pozitif,Yüksek İhtimal


In [64]:
#Argo cümle
model_path ="martin-ha/toxic-comment-model"
tokenizer=AutoTokenizer.from_pretrained(model_path)
model=AutoModelForSequenceClassification.from_pretrained(model_path)
pipeline=TextClassificationPipeline(model=model, tokenizer=tokenizer)
text="bu cümle toksik bir cümle değil amınakoyayım"
b_text=TextBlob(text)
e_text=b_text.translate(from_lang='tr', to='en')  
sonuc=pipeline(str(e_text))
if sonuc[0]['label']=="toxic":
    print ("Toxic")
else:
    print ("Toxic değil")

Toxic değil


In [65]:
#Eksik Cümle
unmasker=pipeline('fill-mask', model='xlm-roberta-base')


TypeError: _batch_encode_plus() got an unexpected keyword argument 'model'